<a href="https://colab.research.google.com/github/cappelchi/calcio_notebooks/blob/main/draft/football_live_experimental_EDA_lines_heft_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Project config

In [ ]:
try:
    import neptune.new as neptune
except:
    !pip install neptune-client >> None
    import neptune.new as neptune
#from neptune.new.integrations.tensorflow_keras import NeptuneCallback
def get_credential(frmwork = 'neptune_team'):
    with open('credential.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw

/usr/local/lib/python3.9/dist-packages/neptune/internal/backends/hosted_client.py:50: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version
<ipython-input-1-94b4efcbc4c0>:5: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  import neptune.new as neptune


In [ ]:
#@title Set API key for neptune.ai
set_api = True #@param {type:"boolean"}
if set_api:
    username, api_key = get_credential()

### Downloads

In [ ]:
 #!wget -q -O ./data2.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/GfaqHfbXoy9kXg
#!unrar e data2.rar train_id.csv test_id.csv ./
#!rm ./data2.rar


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from data2.rar

Extracting  ./test_id.csv                                                  7%  OK 
Extracting  ./train_id.csv                                                 7%  OK 
All OK


In [ ]:
!wget -q -O ./data.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/IWvldpeoQcAh5w
!unrar e ./data.rar

In [ ]:
!wc -l ./data.csv
#14899849 ./data.csv

15228352 ./data.csv


### Imports

In [3]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
print(pd.__version__)
print(np.__version__)

1.5.3
1.22.4


In [ ]:
np.iinfo(np.int8).max, np.iinfo(np.int16).max, np.iinfo(np.int32).max

(127, 32767, 2147483647)

In [ ]:
np.finfo(np.float16).precision, np.finfo(np.float32).precision, np.finfo(np.float64).precision

(3, 6, 15)

In [4]:
import os, psutil
from neptune.utils import stringify_unsupported
from tqdm import tqdm
import gc

### Code

#####Functions

In [ ]:
def run_bash(bashCommand:str, nameCommand = ''):
        process = subprocess.Popen([bashCommand], 
                           shell=True)
        _, error = process.communicate()
        if error:
            print(f'{nameCommand} error:\n', error)

In [ ]:
# Присоединяем итоговый результат и по первому тайму
def add_match_results (data_df:pd.DataFrame, cols = ['Id', 'Result1', 'Result2', 'Periods'], info_path = './info.csv'):
    info_df = pd.read_csv(info_path, sep = ';', usecols = cols)
    info_df[['Period1', 'Period2', 'Period3', 'Period4']] = info_df['Periods'].str.split(',', expand = True)
    info_df[['Time1Res1', 'Time1Res2']] = info_df['Period1'].str.split(':', expand = True)
    info_df = info_df[~info_df['Id'].duplicated(keep = False)]
    period1_result_dict = info_df.set_index('Id')[['Time1Res1', 'Time1Res2', 'Result1', 'Result2']].to_dict(orient = 'index')
    #data_df.loc[:, ['Time1Res1', 'Time1Res2', 'Result1', 'Result2']] =  \
    return [[
        #period1_result_dict[id]['Time1Res1'], period1_result_dict[id]['Time1Res1'], 
        period1_result_dict[id]['Result1'], period1_result_dict[id]['Result2']] 
    if id in period1_result_dict else [None, None] for id in tqdm(data_df['Id'].values, total = len(data_df))]
    

In [ ]:
# Добавляем прематчевые линии
def add_match_lines (data_df:pd.DataFrame, cols = ['P1', 'P2'], prem_path = './prem.csv'):
    prem_df = pd.read_csv(prem_path, sep = ';')
    prem_df = prem_df[~prem_df['Id'].duplicated(keep = False)]
    prem_dict = prem_df.set_index('Id')[cols].to_dict(orient = 'index')
    #data_df.loc[:,['P1', 'PX', 'P2', 'PR']] =  \
    return [[prem_dict[id]['P1'], prem_dict[id]['P2']] 
    if id in prem_dict else [None, None] for id in tqdm(data_df['Id'].values, total = len(data_df))]

In [ ]:
def transform_dataset(data_df:pd.DataFrame, remain = []):
    new_match_vector = data_df['Id'] != data_df['Id'].shift(1)
    # трансформируем минуты
    data_df['min_norm'] = data_df['Minute'].astype(np.float32) / 50
    print('1. Минуты посчитаны...')
    # трансформируем голы
    data_df[data_df['Score1'].isna() & new_match_vector] = 0
    data_df['Score1_norm'] = data_df['Score1'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score1'] > 3, ['Score1_norm']] = 1.0
    data_df[data_df['Score2'].isna() & new_match_vector] = 0
    data_df['Score2_norm'] = data_df['Score2'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score2'] > 3, ['Score2_norm']] = 1.0

    data_df['Score_diff'] = data_df['Score1'].astype(np.int16) - data_df['Score2'].astype(np.int16)
    data_df.loc[data_df['Score_diff'] < -4, ['Score_diff']] = -4
    data_df.loc[data_df['Score_diff'] > 4, ['Score_diff']] = 4
    data_df[[f'Score_cat_{n}' for n in range(1, 10)]] = pd.get_dummies(data_df['Score_diff']).values
    data_df['Score_diff'] = data_df['Score_diff'].astype(np.float32) / np.float32(4.0)
    if 'Score1' not in remain:
        data_df = data_df.drop(['Score1', 'Score2'], axis = 1)
    print('2. Голы посчитаны...')
    #трансформируем атаки
    data_df['A1_scaled'] = data_df['A1'].astype(np.float32) / 75
    data_df.loc[data_df['A1'] >= 60, ['A1_scaled']] = (60 + (data_df['A1'] - 60) / 4) / 75
    data_df['A2_scaled'] = data_df['A2'].astype(np.float32) / 75
    data_df.loc[data_df['A2'] >= 60, ['A2_scaled']] = (60 + (data_df['A2'] - 60) / 4) / 75
    # атаки в минуту
    data_df['A1perMIN'] = data_df['A1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A1perMIN'] > 4, ['A1perMIN']] = np.float32(4.0)
    data_df['A2perMIN'] = data_df['A2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A2perMIN'] > 4, ['A2perMIN']] = np.float32(4.0)
    # динамика атак
    new_match_vector5 = data_df['Id'] != data_df['Id'].shift(5)
    data_df['A1relativ'] = data_df['A1'].astype(np.float32) - data_df['A1'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['A1relativ']] = np.float32(0.0)
    data_df['A1relativ'] = data_df['A1relativ'].fillna(0)
    data_df.loc[data_df['A1relativ'] > 15, ['A1relativ']] = np.float32(15.)
    data_df['A2relativ'] = data_df['A2'].astype(np.float32) - data_df['A2'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['A2relativ']] =  np.float32(0.0)
    data_df['A2relativ'] = data_df['A2relativ'].fillna(0)
    data_df.loc[data_df['A2relativ'] > 15, ['A2relativ']] = np.float32(15.)
    if 'A1' not in remain:
        data_df = data_df.drop(['A1', 'A2'], axis = 1)
    print('3. Атаки посчитаны...')
    # трансформируем опасные атаки
    data_df['DA1_scaled'] = data_df['DA1'].astype(np.float32) / 50
    data_df.loc[data_df['DA1'] >= 40, ['DA1_scaled']] = (80 + (data_df['DA1'] - 40) / 3) / 100
    data_df['DA2_scaled'] = data_df['DA2'].astype(np.float32) / 50
    data_df.loc[data_df['DA2'] >= 40, ['DA2_scaled']] = (80 + (data_df['DA2'] - 40) / 3) / 100
    # опасные атаки в минуту    
    data_df['DA1perMIN'] = data_df['DA1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA1perMIN'] > 3, ['DA1perMIN']] = np.float32(3.0)
    data_df['DA2perMIN'] = data_df['DA2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA2perMIN'] > 3, ['DA2perMIN']] = np.float32(3.0)
    # динамика опасных атак
    data_df['DA1relativ'] = data_df['DA1'].astype(np.float32) - data_df['DA1'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['DA1relativ']] = np.float32(0.0)
    data_df['DA1relativ'] = data_df['DA1relativ'].fillna(0)
    data_df.loc[data_df['DA1relativ'] > 10, ['DA1relativ']] = np.float32(10.)
    data_df['DA2relativ'] = data_df['DA2'].astype(np.float32) - data_df['DA2'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['DA2relativ']] = np.float32(0.0)
    data_df['DA2relativ'] = data_df['DA2relativ'].fillna(0)
    data_df.loc[data_df['DA2relativ'] > 10, ['DA2relativ']] = np.float32(10.)
    if 'DA1' not in remain:
        data_df = data_df.drop(['DA1', 'DA2'], axis = 1)
    if 'Minute' not in remain:
        data_df = data_df.drop(['Minute'], axis = 1)
    print('4. Опасные атаки посчитаны...')
    # трансформируем владение мячом
    data_df[data_df['Pos1'].isna() & new_match_vector] = 0
    data_df['Pos1_cleaned'] = data_df['Pos1'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos1_cleaned'] < 0.2, ['Pos1_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos1_cleaned'] > 0.8, ['Pos1_cleaned']] = np.float32(0.8)
    data_df[data_df['Pos2'].isna() & new_match_vector] = 0
    data_df['Pos2_cleaned'] = data_df['Pos2'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos2_cleaned'] < 0.2, ['Pos2_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos2_cleaned'] > 0.8, ['Pos2_cleaned']] = np.float32(0.8)
    if 'Pos1' not in remain:
        data_df = data_df.drop(['Pos1', 'Pos2'], axis = 1)
    print('5. Владение мячом посчитпно...')
    # трансформируем удары
    data_df[data_df['Off1'].isna() & new_match_vector] = 0
    data_df['Off1_norm'] = data_df['Off1'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off1_norm'] > 1.0, ['Off1_norm']] = np.float32(1.0)
    data_df[data_df['Off2'].isna() & new_match_vector] = 0
    data_df['Off2_norm'] = data_df['Off2'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off2_norm'] > 1.0, ['Off2_norm']] = np.float32(1.0)
    if 'Off1' not in remain:
        data_df = data_df.drop(['Off1', 'Off2'], axis = 1)
    print('6. Удары посчитаны...')
    # трансформируем удары в створ
    data_df[data_df['On1'].isna() & new_match_vector] = 0
    data_df['On1_norm'] = data_df['On1'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On1_norm'] > 1.0, ['On1_norm']] = np.float32(1.0)
    data_df[data_df['On2'].isna() & new_match_vector] = 0
    data_df['On2_norm'] = data_df['On2'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On2_norm'] > 1.0, ['On2_norm']] = np.float32(1.0)
    if 'On1' not in remain:
        data_df = data_df.drop(['On1', 'On2'], axis = 1)    
    print('7. Удары в створ посчитаны...')
    # трансформируем желтые карточки
    data_df[data_df['YC1'].isna() & new_match_vector] = 0
    data_df['YC1_transformed'] = data_df['YC1'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC1_transformed'] > 1.0, ['YC1_transformed']] = np.float32(1.0)
    data_df[data_df['YC2'].isna() & new_match_vector] = 0
    data_df['YC2_transformed'] = data_df['YC2'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC2_transformed'] > 1.0, ['YC2_transformed']] = np.float32(1.0)
    if 'YC1' not in remain:
        data_df = data_df.drop(['YC1', 'YC2'], axis = 1)
    print('8. Жёлтые карточки посчитаны...')
    # трансформируем красные карточки
    data_df[data_df['RC1'].isna() & new_match_vector] = 0
    data_df['RC1_transformed'] = data_df['RC1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC1_transformed'] > 1, ['RC1_transformed']] = np.int8(1)
    data_df[data_df['RC2'].isna() & new_match_vector] = 0
    data_df['RC2_transformed'] = data_df['RC2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC2_transformed'] > 1, ['RC2_transformed']] = np.int8(1)
    if 'RC1' not in remain:
        data_df = data_df.drop(['RC1', 'RC2'], axis = 1)
    print('9. Красные карточки посчитаны...')
    # трансформируем замены
    data_df[data_df['Sub1'].isna() & new_match_vector] = 0
    data_df['Sub1_transformed'] = data_df['Sub1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub1_transformed'] > 1, ['Sub1_transformed']] = np.int8(1)
    data_df[data_df['Sub2'].isna() & new_match_vector] = 0
    data_df['Sub2_transformed'] = data_df['Sub2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub2_transformed'] > 1, ['Sub2_transformed']] = np.int8(1)
    if 'Sub1' not in remain:
        data_df = data_df.drop(['Sub1', 'Sub2'], axis = 1)
    print('10. Замены посчитаны...')
    # трансформируем угловык
    data_df[data_df['Cor1'].isna() & new_match_vector] = 0
    data_df['Cor1_transformed'] = data_df['Cor1'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor1_transformed'] > 1.0, ['Cor1_transformed']] = np.float32(1.0)
    data_df[data_df['Cor2'].isna() & new_match_vector] = 0
    data_df['Cor2_transformed'] = data_df['Cor2'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor2_transformed'] > 1.0, ['Cor2_transformed']] = np.float32(1.0)
    if 'Cor1' not in remain:
        data_df = data_df.drop(['Cor1', 'Cor2'], axis = 1)
    print('11. Угловые посчитаны...')
    # трансформируем линию
    data_df['P1_transformed'] = np.log(data_df['P1'], dtype = np.float32) / 2
    data_df['P2_transformed'] = np.log(data_df['P2'], dtype = np.float32) / 2
    if 'P1' not in remain:
        data_df = data_df.drop(['P1', 'P2'], axis = 1)
    print('12. Линии посчитаны...')
    if 'Pen1' not in remain:
        data_df = data_df.drop(['Pen1', 'Pen2'], axis = 1)
    if 'Active' not in remain:
        data_df = data_df.drop(['Active'], axis = 1)
    return data_df

#### Predefine data

In [ ]:
data_types_dict = {
    'Id':np.int32, 
    'StatTime':np.datetime64, 
    'Minute':np.int8, 
    'Active': np.int8, 'Score1':np.int8, 'Score2':np.int8,
    'A1':np.int16, 'A2':np.int16, 'DA1':np.int16, 'DA2':np.int16, 'Pos1':np.float32, 'Pos2':np.float32,
    'Off1':np.int8, 'Off2':np.int8, 'On1':np.int8, 'On2':np.int8, 'YC1':np.int8, 'YC2':np.int8,
    'RC1':np.int8, 'RC2':np.int8, 'Sub1':np.int8, 'Sub2':np.int8, 'Pen1':np.int8, 'Pen2':np.int8,
    'Cor1':np.int8, 'Cor2':np.int8, 'Period':np.int8, 
    'D':np.datetime64,
    'I':np.int32, 'Active.1':np.int8, 
    'Time':np.datetime64, 
    'Minute.1':np.int8, 
    'RawTime':np.datetime64,
    'Score1.1':np.int8, 'Score2.1':np.int8, 'Period.1':np.int8, 
    'W1':np.float16, 'WX':np.float16, 'W2':np.float16, 'X1':np.float16, 'X2':np.float16, 'W12':np.float16, 'TotalValue':np.float16, 
    'Over':np.float16, 'Under':np.float16, 'Hand1Value':np.float16, 'H1':np.float16, 'H2':np.float16 
}

In [ ]:
drop_cols = [
    #'StatTime', 
    'Comment', 'D', 'I', 'Time', 'Minute.1', 'RawTime',
    'Score1.1', 'Score2.1', 'Period.1', 'Period', 'Periods', 'Serve'
]

In [ ]:
k_cols = [
    'W1', 'WX', 'W2', 'X1', 'X2', 'W12', 'TotalValue' ,'Over',
    'Under', 'Hand1Value', 'H1', 'H2' 
]

In [ ]:
test_dict = {
    6015559:2, 6053875:3, 6127595:2, 6206288:11, 6368081:3, 6015561:1,
    6054180:3, 6147868:3, 6348541:2, 6015562:2, 6055136:3, 6150224:3, 6354196:2
}
#vector = pd.read_csv('./data.csv', sep = ';', usecols = ['Id'])
#vector_bool = vector['Id'].isin(test_dict)

#### create dataframe

In [ ]:
#data_df = pd.read_csv('/content/data.csv', sep = ';', usecols = ['Id', 'StatTime'], parse_dates = ['StatTime'])

In [ ]:
#pd.DataFrame(
#    data_df['Id'][data_df['StatTime'].str.contains('2023')].unique()
#    ).to_csv('./holdout_id.csv', header = None, index = None)
#	6506727

In [ ]:
data_df = pd.DataFrame()

In [ ]:
for col, col_type in data_types_dict.items(): #{'Id':np.int32 , 'StatTime':np.datetime64}.items(): #
    if col not in drop_cols:
        
        if col == 'Id':
            data_col = pd.read_csv('./data.csv', sep = ';', usecols = [col])
            if data_col[col].isna().sum() > 0:
                print(col , ' nan error')
            data_col = data_col.astype(col_type)
            new_match_vector = data_col['Id'] != data_col['Id'].shift(1)
            data_df[col] = data_col
        elif col == 'StatTime':
            data_col = pd.read_csv('./data.csv', sep = ';', 
                                   usecols = ['StatTime'], 
                                   parse_dates = ['StatTime'])
            data_col.loc[data_col['StatTime'] == 'NA                 ',['StatTime']] = '31.01.2010 00:00:00'
            #data_col = data_col.astype(col_type)
            data_df[col] = data_col
        elif col in k_cols:
            data_col = pd.read_csv('./data.csv', sep = ';', usecols = [col])
            data_col = data_col.fillna(0).astype(col_type)
            data_df[col] = data_col
        else:
            data_col = pd.read_csv('./data.csv', sep = ';', usecols = [col])
            data_col[data_col[col].isna() & new_match_vector] = 0
            data_col = data_col.fillna(method = 'ffill').astype(col_type)
            data_df[col] = data_col
        del data_col
        gc.collect()
        #---------------
        process = psutil.Process(os.getpid())
        print(col, ' mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
        #---------------   
    #if col == 'Active':
    #    break

Id  mem usage:  0.45 GiB
StatTime  mem usage:  1.32 GiB
Minute  mem usage:  1.21 GiB
Active  mem usage:  1.21 GiB
Score1  mem usage:  1.22 GiB
Score2  mem usage:  1.25 GiB
A1  mem usage:  1.27 GiB
A2  mem usage:  1.27 GiB
DA1  mem usage:  1.29 GiB
DA2  mem usage:  1.44 GiB
Pos1  mem usage:  1.38 GiB
Pos2  mem usage:  1.44 GiB
Off1  mem usage:  1.44 GiB
Off2  mem usage:  1.62 GiB
On1  mem usage:  1.44 GiB
On2  mem usage:  1.45 GiB
YC1  mem usage:  1.47 GiB
YC2  mem usage:  1.48 GiB
RC1  mem usage:  1.68 GiB
RC2  mem usage:  1.51 GiB
Sub1  mem usage:  1.52 GiB
Sub2  mem usage:  1.54 GiB
Pen1  mem usage:  1.55 GiB
Pen2  mem usage:  1.75 GiB
Cor1  mem usage:  1.69 GiB
Cor2  mem usage:  1.67 GiB
Active.1  mem usage:  1.79 GiB
W1  mem usage:  1.7 GiB
WX  mem usage:  1.85 GiB
W2  mem usage:  1.88 GiB
X1  mem usage:  1.79 GiB
X2  mem usage:  1.82 GiB
W12  mem usage:  1.85 GiB
TotalValue  mem usage:  1.88 GiB
Over  mem usage:  2.02 GiB
Under  mem usage:  2.05 GiB
Hand1Value  mem usage:  1.96 Gi

#### create time.csv

In [ ]:
(data_df['Id'].astype(str) + ':' + data_df['StatTime'].astype(str).str.split(
        ' ', expand=True)[0]
        ).drop_duplicates().str.split(':', expand = True).rename(
                                                columns = {0:'Id', 1:'StatTime'}
                                                            ).to_csv('./time.csv', index = False)
data_df = data_df.drop(['StatTime'], axis = 1)

In [ ]:
len(data_df)

15228351

In [ ]:
#time_df = data_df.drop_duplicates()
#time_df.to_csv('./time.csv', index = False)

#### add final results

In [ ]:
## Добавляем финальный результат
data_df.loc[:, ['Result1', 'Result2']] = add_match_results (data_df)
print('\n Nan: ', data_df['Result1'].isna().sum())
data_df = data_df[~data_df['Result1'].isna()]
data_df[['Result1', 'Result2']] = data_df[['Result1', 'Result2']].astype(np.int8)

## Добавляем линии
data_df.loc[:,['P1', 'P2']] = add_match_lines (data_df)
print('\n Nan: ', data_df['P1'].isna().sum())
data_df = data_df[~data_df['P1'].isna()]
data_df[['P1', 'P2']] = data_df[['P1', 'P2']].astype(np.float16)
#---------------
process = psutil.Process(os.getpid())
print(col, ' mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
#---------------    

100%|██████████| 15228351/15228351 [01:55<00:00, 131869.45it/s]



 Nan:  108


100%|██████████| 15228243/15228243 [01:53<00:00, 134499.37it/s]



 Nan:  107056
H2  mem usage:  1.56 GiB


In [ ]:
data_df.tail()

,Id,Minute,Active,Score1,Score2,A1,A2,DA1,DA2,Pos1,Pos2,Off1,Off2,On1,On2,YC1,YC2,RC1,RC2,Sub1,Sub2,Pen1,Pen2,Cor1,Cor2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2,Result1,Result2,P1,P2
15228346,6506727,39,1,1,0,44,33,24,12,0.0,0.0,5,0,1,4,0,2,0,0,0,1,0,0,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,1.200195,15.0
15228347,6506727,40,1,1,0,47,33,25,14,0.0,0.0,5,1,1,4,0,2,0,0,0,1,0,0,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,1.200195,15.0
15228348,6506727,41,1,1,0,50,33,26,14,0.0,0.0,5,1,1,4,0,2,0,0,0,1,0,0,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,1.200195,15.0
15228349,6506727,42,1,1,0,50,34,26,14,0.0,0.0,5,1,1,4,0,2,0,0,0,1,0,0,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,1.200195,15.0
15228350,6506727,45,1,1,0,51,34,26,14,0.0,0.0,5,1,1,4,0,2,0,0,0,1,0,0,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,1.200195,15.0


#### transform data

In [ ]:
data_df = transform_dataset(data_df, #) 
    remain = ['Score1', 'Score2'])

1. Минуты посчитаны...
2. Голы посчитаны...
3. Атаки посчитаны...
4. Опасные атаки посчитаны...
5. Владение мячом посчитпно...
6. Удары посчитаны...
7. Удары в створ посчитаны...
8. Жёлтые карточки посчитаны...
9. Красные карточки посчитаны...
10. Замены посчитаны...
11. Угловые посчитаны...


/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


12. Линии посчитаны...


#### delete NaN

In [ ]:
print('P1 NaN & P2 NaN: ', data_df['P1_transformed'].isna().sum(), data_df['P2_transformed'].isna().sum())

data_df = data_df[~data_df['P1_transformed'].isna()]
data_df = data_df[~data_df['P2_transformed'].isna()]

print('Score-Result error 1&2: ', (~(data_df['Result1'] - data_df['Score1']) >= 0).sum(), (~(data_df['Result1'] - data_df['Score1']) >= 0).sum())
data_df = data_df.loc[(data_df['Result1'] - data_df['Score1']) >= 0]
data_df = data_df.loc[(data_df['Result2'] - data_df['Score2']) >= 0]


#---------------
process = psutil.Process(os.getpid())
print('mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes 
#---------------    

P1 NaN & P2 NaN:  0 0
Score-Result error 1&2:  856 856
mem usage:  7.07 GiB


#### some results

In [ ]:
data_df.loc[(data_df['Result1'] - data_df['Score1']) < 0]

,Id,Score1,Score2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2,Result1,Result2,min_norm,Score1_norm,Score2_norm,Score_diff,Score_cat_1,Score_cat_2,Score_cat_3,Score_cat_4,Score_cat_5,Score_cat_6,Score_cat_7,Score_cat_8,Score_cat_9,A1_scaled,A2_scaled,A1perMIN,A2perMIN,A1relativ,A2relativ,DA1_scaled,DA2_scaled,DA1perMIN,DA2perMIN,DA1relativ,DA2relativ,Pos1_cleaned,Pos2_cleaned,Off1_norm,Off2_norm,On1_norm,On2_norm,YC1_transformed,YC2_transformed,RC1_transformed,RC2_transformed,Sub1_transformed,Sub2_transformed,Cor1_transformed,Cor2_transformed,P1_transformed,P2_transformed


In [ ]:
((data_df['Result1'] - data_df['Score1']) < 0).sum()

0

In [ ]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15119802 entries, 0 to 15228350
Data columns (total 59 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Id                int32  
 1   Score1            int8   
 2   Score2            int8   
 3   Active.1          int8   
 4   W1                float16
 5   WX                float16
 6   W2                float16
 7   X1                float16
 8   X2                float16
 9   W12               float16
 10  TotalValue        float16
 11  Over              float16
 12  Under             float16
 13  Hand1Value        float16
 14  H1                float16
 15  H2                float16
 16  Result1           int8   
 17  Result2           int8   
 18  min_norm          float32
 19  Score1_norm       float32
 20  Score2_norm       float32
 21  Score_diff        float32
 22  Score_cat_1       uint8  
 23  Score_cat_2       uint8  
 24  Score_cat_3       uint8  
 25  Score_cat_4       uint8  
 26  Score_cat_5 

In [ ]:
data_df.describe()

,Id,Score1,Score2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2,Result1,Result2,min_norm,Score1_norm,Score2_norm,Score_diff,Score_cat_1,Score_cat_2,Score_cat_3,Score_cat_4,Score_cat_5,Score_cat_6,Score_cat_7,Score_cat_8,Score_cat_9,A1_scaled,A2_scaled,A1perMIN,A2perMIN,A1relativ,A2relativ,DA1_scaled,DA2_scaled,DA1perMIN,DA2perMIN,DA1relativ,DA2relativ,Pos1_cleaned,Pos2_cleaned,Off1_norm,Off2_norm,On1_norm,On2_norm,YC1_transformed,YC2_transformed,RC1_transformed,RC2_transformed,Sub1_transformed,Sub2_transformed,Cor1_transformed,Cor2_transformed,P1_transformed,P2_transformed
count,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,15119802.0,1.511980e+07,1.511980e+07,15119802.0,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07,1.511980e+07
mean,3.806749e+06,3.131681e-01,2.491339e-01,3.407605e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.569513e+00,1.271828e+00,4.581242e-01,7.812629e-02,6.218962e-02,1.594242e-02,9.387028e-04,3.834905e-03,2.124241e-02,1.261058e-01,6.506366e-01,1.569092e-01,3.208256e-02,6.453920e-03,1.795923e-03,3.327270e-01,3.182437e-01,1.129196e+00,1.079529e+00,4.988400e+00,4.766802e+00,2.749791e-01,2.487535e-01,6.007668e-01,5.416068e-01,2.859596e+00,2.587040e+00,4.564157e-01,4.328656e-01,1.426817e-01,1.237993e-01,2.010737e-01,1.705206e-01,1.133528e-01,1.225857e-01,5.708210e-03,6.675881e-03,2.880249e-02,2.953141e-02,1.927124e-01,1.657641e-01,-inf,-inf
std,1.315812e+06,6.144587e-01,5.419506e-01,4.739650e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,1.460551e+00,1.303003e+00,2.596625e-01,1.523816e-01,1.346734e-01,1.982110e-01,3.062387e-02,6.180775e-02,1.441914e-01,3.319685e-01,4.767689e-01,3.637151e-01,1.762194e-01,8.007664e-02,4.234026e-02,2.096327e-01,2.023994e-01,4.463000e-01,4.370320e-01,3.118670e+00,3.022666e+00,2.032851e-01,1.904377e-01,3.326157e-01,3.190521e-01,2.297457e+00,2.188021e+00,1.649117e-01,1.568069e-01,1.618986e-01,1.477034e-01,2.445237e-01,2.238836e-01,2.450550e-01,2.543316e-01,7.533675e-02,8.143288e-02,1.672510e-01,1.692906e-01,2.275883e-01,2.096369e-01,NaN,NaN
min,3.246400e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,-17.5,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e-02,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e-01,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-inf,-inf
25%,2.704543e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.400000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.600000e-01,1.466667e-01,8.500000e-01,8.064516e-01,3.000000e+00,3.000000e+00,1.200000e-01,1.000000e-01,3.793103e-01,3

In [ ]:
#data_df.to_csv('./data_all.csv', index = False)

In [ ]:
#data_df = pd.read_csv('./data_all.csv',)

In [ ]:
data_df.columns

Index(['Id', 'Score1', 'Score2', 'Active.1', 'W1', 'WX', 'W2', 'X1', 'X2',
       'W12', 'TotalValue', 'Over', 'Under', 'Hand1Value', 'H1', 'H2',
       'Result1', 'Result2', 'min_norm', 'Score1_norm', 'Score2_norm',
       'Score_diff', 'Score_cat_1', 'Score_cat_2', 'Score_cat_3',
       'Score_cat_4', 'Score_cat_5', 'Score_cat_6', 'Score_cat_7',
       'Score_cat_8', 'Score_cat_9', 'A1_scaled', 'A2_scaled', 'A1perMIN',
       'A2perMIN', 'A1relativ', 'A2relativ', 'DA1_scaled', 'DA2_scaled',
       'DA1perMIN', 'DA2perMIN', 'DA1relativ', 'DA2relativ', 'Pos1_cleaned',
       'Pos2_cleaned', 'Off1_norm', 'Off2_norm', 'On1_norm', 'On2_norm',
       'YC1_transformed', 'YC2_transformed', 'RC1_transformed',
       'RC2_transformed', 'Sub1_transformed', 'Sub2_transformed',
       'Cor1_transformed', 'Cor2_transformed', 'P1_transformed',
       'P2_transformed'],
      dtype='object')

In [ ]:
data_df

,Id,Score1,Score2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2,Result1,Result2,min_norm,Score1_norm,Score2_norm,Score_diff,Score_cat_1,Score_cat_2,Score_cat_3,Score_cat_4,Score_cat_5,Score_cat_6,Score_cat_7,Score_cat_8,Score_cat_9,A1_scaled,A2_scaled,A1perMIN,A2perMIN,A1relativ,A2relativ,DA1_scaled,DA2_scaled,DA1perMIN,DA2perMIN,DA1relativ,DA2relativ,Pos1_cleaned,Pos2_cleaned,Off1_norm,Off2_norm,On1_norm,On2_norm,YC1_transformed,YC2_transformed,RC1_transformed,RC2_transformed,Sub1_transformed,Sub2_transformed,Cor1_transformed,Cor2_transformed,P1_transformed,P2_transformed
0,330509,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.02,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.000000,0.066667,0.000000,4.000000,0.0,0.0,0.00,0.04,0.000000,2.000000,0.0,0.0,0.2,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.000000,1.125646,0.155305
1,330509,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.04,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.000000,0.066667,0.000000,2.500000,0.0,0.0,0.00,0.06,0.000000,1.500000,0.0,0.0,0.2,0.8,0.0,0.1,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.000000,1.125646,0.155305
2,330509,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.06,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.026667,0.133333,0.666667,3.333333,0.0,0.0,0.00,0.10,0.000000,1.666667,0.0,0.0,0.2,0.8,0.0,0.1,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.000000,1.125646,0.155305
3,330509,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.08,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.066667,0.146667,1.250000,2.750000,0.0,0.0,0.02,0.10,0.250000,1.250000,0.0,0.0,0.2,0.8,0.0,0.1,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.000000,1.125646,0.155305
4,330509,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.10,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.080000,0.200000,1.200000,3.000000,0.0,0.0,0.04,0.12,0.400000,1.200000,0.0,0.0,0.2,0.8,0.0,0.1,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.000000,1.125646,0.155305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15228346,6506727,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.78,0.25,0.0,0.25,0,0,0,0,0,1,0,0,0,0.586667,0.440000,1.128205,0.846154,4.0,4.0,0.48,0.24,0.615385,0.307692,1.0,1.0,0.2,0.2,0.5,0.0,0.2,0.8,0.0,1.0,0,0,0,1,0.5,0.333333,0.091242,1.354025
15228347,6506727,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.80,0.25,0.0,0.25,0,0,0,0,0,1,0,0,0,0.626667,0.440000,1.175000,0.825000,6.0,3.0,0.50,0.28,0.625000,0.350000,2.0,3.0,0.2,0.2,0.5,0.1,0.2,0.8,0.0,1.0,0,0,0,1,0.5,0.333333,0.091242,1.354025
15228348,6506727,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.82,0.25,0.0,0.25,0,0,0,0,0,1,0,0,0,0.666667,0.440000,1.219512,0.804878,9.0,2.0,0.52,0.28,0.634146,0.341463,3.0,3.0,0.2,0.2,0.5,0.1,0.2,0.8,0.0,1.0,0,0,0,1,0.5,0.333333,0.091242,1.354025
15228349,6506727,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.84,0.25,0.0,0.25,0,0,0,0,0,1,0,0,0,0.666667,0.453333,1.190476,0.809524,8.0,3.0,0.52,0.28,0.619048,0.333333,3.0,3.0,0.2,0.2,0.5,0.1,0.2,0.8,0.0,1.0,0,0,0,1,0.5,0.333333,0.091242,1.354025


In [ ]:
#prod_test_df = data_df[data_df['Id'].isin(test_dict)]
#prod_test_vector = data_df['Id'].isin(test_dict)
#prod_test_df.to_csv('./prod_test.csv', index = False)

In [ ]:
#np.savez_compressed('./prod_test', 
#                    #id_train = id_train,
#                    #id_test = id_test,
#                    X_prod_test =  data_df[data_df.columns[18:]][prod_test_vector].values,
#                    y_prod_test_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[prod_test_vector],
#                    y_train_multi = np.sign((data_df['Result1'] - data_df['Result2']).values)[prod_test_vector] + 1,
#                    y_train_diff = (data_df['Result1'] - data_df['Result2']).values[prod_test_vector],
#                    y_train_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[prod_test_vector],
#                    y_train_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[prod_test_vector],
#                    score1_train = data_df['Score1'].values[prod_test_vector],
#                    score2_train = data_df['Score2'].values[prod_test_vector],
#                    result1_train = data_df['Result1'].values[prod_test_vector],
#                    result2_train = data_df['Result2'].values[prod_test_vector],
#                    K_train = data_df[data_df.columns[3:16]].values[prod_test_vector]
#                )

#### split data & prepare files for upload

In [ ]:
cols = ['Id', 'BeginTime']
info_path = './info.csv'
info_df = pd.read_csv(info_path, sep = ';', 
                      usecols = cols
                      )


In [ ]:
train_id_df = pd.read_csv('./train_id.csv', header = None, names = ['id'])
test_id_df = pd.read_csv('./test_id.csv', header = None, names = ['id'])
holdout_id_df = pd.read_csv('./holdout_id.csv', header = None, names = ['id'])

train_vector = data_df['Id'].isin(train_id_df['id'])
test_vector = data_df['Id'].isin(test_id_df['id'])
holdout_vector = data_df['Id'].isin(holdout_id_df['id'])

In [ ]:
data_df[data_df['Id'] == 6506727]

,Id,Score1,Score2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2,Result1,Result2,min_norm,Score1_norm,Score2_norm,Score_diff,Score_cat_1,Score_cat_2,Score_cat_3,Score_cat_4,Score_cat_5,Score_cat_6,Score_cat_7,Score_cat_8,Score_cat_9,A1_scaled,A2_scaled,A1perMIN,A2perMIN,A1relativ,A2relativ,DA1_scaled,DA2_scaled,DA1perMIN,DA2perMIN,DA1relativ,DA2relativ,Pos1_cleaned,Pos2_cleaned,Off1_norm,Off2_norm,On1_norm,On2_norm,YC1_transformed,YC2_transformed,RC1_transformed,RC2_transformed,Sub1_transformed,Sub2_transformed,Cor1_transformed,Cor2_transformed,P1_transformed,P2_transformed
15228311,6506727,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.02,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.013333,0.013333,1.000000,1.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.000000,0.000000,0.091242,1.354025
15228312,6506727,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.04,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.026667,0.053333,1.000000,2.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.000000,0.000000,0.091242,1.354025
15228313,6506727,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.06,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.040000,0.053333,1.000000,1.333333,0.0,0.0,0.02,0.00,0.333333,0.000000,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.166667,0.000000,0.091242,1.354025
15228314,6506727,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.08,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.040000,0.053333,0.750000,1.000000,0.0,0.0,0.06,0.00,0.750000,0.000000,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.333333,0.000000,0.091242,1.354025
15228315,6506727,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.10,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.066667,0.093333,1.000000,1.400000,0.0,0.0,0.06,0.02,0.600000,0.200000,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0,0,0,0,0.333333,0.000000,0.091242,1.354025
15228316,6506727,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.12,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.093333,0.093333,1.166667,1.166667,6.0,6.0,0.10,0.02,0.833333,0.166667,5.0,1.0,0.2,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0,0,0,0,0.333333,0.000000,0.091242,1.354025
15228317,6506727,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.14,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.106667,0.133333,1.142857,1.428571,6.0,6.0,0.10,0.02,0.714286,0.142857,5.0,1.0,0.2,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0,0,0,0,0.333333,0.000000,0.091242,1.354025
15228318,6506727,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.16,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.133333,0.133333,1.250000,1.250000,7.0,6.0,0.10,0.02,0.625000,0.125000,4.0,1.0,0.2,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0,0,0,0,0.333333,0.000000,0.091242,1.354025
15228319,6506727,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.18,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.160000,0.146667,1.333333,1.222222,9.0,7.0,0.12,0.06,0.666667,0.333333,3.0,3.0,0.2,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0,0,0,0,0.333333,0.000000,0.091242,1.354025
15228320,6506727,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0.20,0.00,0.0,0.00,0,0,0,0,1,0,0,0,0,0.173333,0.160000,1.300000,1.200000,8.0,5.0,0.14,0.06,0.700000,0.300000,4.0,2.0,0.2,0.2,0.1,0.0,0.0,0.2,0.0,0.0,0,0,0,0,0.333333,0.000000,0.091242,1.354025


In [ ]:
#%%writefile save_discription.txt
np.savez_compressed('./dataset', 
                    #id_train = id_train,
                    #id_test = id_test,
                    X_train =  data_df[data_df.columns[18:]][train_vector].values,
                    X_test = data_df[data_df.columns[18:]][test_vector].values,
                    X_holdout = data_df[data_df.columns[18:]][holdout_vector].values,
                    y_train_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[train_vector],
                    y_test_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[test_vector],
                    y_holdout_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[holdout_vector],
                    y_train_multi = np.sign((data_df['Result1'] - data_df['Result2']).values)[train_vector] + 1,
                    y_test_multi = np.sign((data_df['Result1'] - data_df['Result2']).values)[test_vector] + 1,
                    y_holdout_multi = np.sign((data_df['Result1'] - data_df['Result2']).values)[holdout_vector] + 1,
                    y_train_diff = (data_df['Result1'] - data_df['Result2']).values[train_vector],
                    y_test_diff = (data_df['Result1'] - data_df['Result2']).values[test_vector],
                    y_holdout_diff = (data_df['Result1'] - data_df['Result2']).values[holdout_vector],
                    y_train_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[train_vector],
                    y_train_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[train_vector],
                    y_test_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[test_vector],
                    y_test_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[test_vector],
                    y_holdout_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[holdout_vector],
                    y_holdout_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[holdout_vector],
                    score1_train = data_df['Score1'].values[train_vector],
                    score2_train = data_df['Score2'].values[train_vector],
                    result1_train = data_df['Result1'].values[train_vector],
                    result2_train = data_df['Result2'].values[train_vector],
                    score1_test = data_df['Score1'].values[test_vector],
                    score2_test = data_df['Score2'].values[test_vector],
                    result1_test = data_df['Result1'].values[test_vector],
                    result2_test = data_df['Result2'].values[test_vector],
                    score1_holdout = data_df['Score1'].values[holdout_vector],
                    score2_holdout = data_df['Score2'].values[holdout_vector],
                    result1_holdout = data_df['Result1'].values[holdout_vector],
                    result2_holdout = data_df['Result2'].values[holdout_vector],
                    K_train = data_df[data_df.columns[3:16]].values[train_vector],
                    K_test = data_df[data_df.columns[3:16]].values[test_vector],
                    K_holdout = data_df[data_df.columns[3:16]].values[holdout_vector]
                )

Writing save_discription.txt


In [ ]:
np.savez_compressed('./additional_data', 
                    id_train = data_df['Id'][train_vector].values,
                    id_test = data_df['Id'][test_vector].values,
                    id_holdout = data_df['Id'][holdout_vector].values,
                    min_train = data_df['min_norm'][train_vector].values,
                    min_test = data_df['min_norm'][test_vector].values,
                    min_holdout = data_df['min_norm'][holdout_vector].values
)

In [ ]:
params = {}
params['description'] = 'обновленный датасет https://disk.yandex.ru/d/t3GXEat3tuZ36w \n' + \
'4 класса задач, бинарная классификация и мультикласс, и под регрессию 2 варианта '
params['features'] = list(data_df.columns)

#### upload data to neptune.ai

In [ ]:
data_version = 'football_live_npz_230410/'
project = neptune.init_project(
    project="scomesse/football", 
    api_token = api_key
    )
project[data_version + 'dataset'].upload('./dataset.npz')
project[data_version + 'description'].upload('./save_discription.txt')
project[data_version + 'additional_data'].upload('./additional_data.npz')
project[data_version + 'info'].upload('./info.csv')
project[data_version + 'time'].upload('./time.csv')
project[data_version + 'params'] = stringify_unsupported(params)
project.stop()

https://app.neptune.ai/scomesse/football/
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 7 operations to synchronize with Neptune. Do not kill this process.
All 7 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/metadata


### draft

In [ ]:
data_df.columns[:18]

Index(['Id', 'Score1', 'Score2', 'Active.1', 'W1', 'WX', 'W2', 'X1', 'X2',
       'W12', 'TotalValue', 'Over', 'Under', 'Hand1Value', 'H1', 'H2',
       'Result1', 'Result2'],
      dtype='object')

In [ ]:
data_df.columns

Index(['Id', 'Score1', 'Score2', 'Active.1', 'W1', 'WX', 'W2', 'X1', 'X2',
       'W12', 'TotalValue', 'Over', 'Under', 'Hand1Value', 'H1', 'H2',
       'Result1', 'Result2', 'min_norm', 'Score1_norm', 'Score2_norm',
       'Score_diff', 'Score_cat_1', 'Score_cat_2', 'Score_cat_3',
       'Score_cat_4', 'Score_cat_5', 'Score_cat_6', 'Score_cat_7',
       'Score_cat_8', 'Score_cat_9', 'A1_scaled', 'A2_scaled', 'A1perMIN',
       'A2perMIN', 'A1relativ', 'A2relativ', 'DA1_scaled', 'DA2_scaled',
       'DA1perMIN', 'DA2perMIN', 'DA1relativ', 'DA2relativ', 'Pos1_cleaned',
       'Pos2_cleaned', 'Off1_norm', 'Off2_norm', 'On1_norm', 'On2_norm',
       'YC1_transformed', 'YC2_transformed', 'RC1_transformed',
       'RC2_transformed', 'Sub1_transformed', 'Sub2_transformed',
       'Cor1_transformed', 'Cor2_transformed', 'P1_transformed',
       'P2_transformed'],
      dtype='object')

In [ ]:
X_train = data_df[data_df.columns[18:]][train_vector].values
X_test = data_df[data_df.columns[18:]][test_vector].values
X_holdout = data_df[data_df.columns[18:]][holdout_vector].values

In [ ]:
y_train_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[train_vector]
y_test_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[test_vector]
y_holdout_bin = np.array((data_df['Result1'] > data_df['Result2']) *1, dtype = np.int8)[holdout_vector])

In [ ]:
y_train_multi = np.sign((data_df['Result1'] - data_df['Result2']).values)[train_vector] + 1
y_test_multi = np.sign((data_df['Result1'] - data_df['Result2']).values)[test_vector] + 1
y_holdout_multi = np.sign((data_df['Result1'] - data_df['Result2']).values)[holdout_vector] + 1

In [ ]:
y_train_diff = (data_df['Result1'] - data_df['Result2']).values[train_vector]
y_test_diff = (data_df['Result1'] - data_df['Result2']).values[test_vector]
y_holdout_diff = (data_df['Result1'] - data_df['Result2']).values[holdout_vector]

In [ ]:
y_train_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[train_vector]
y_train_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[train_vector]
y_test_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[test_vector]
y_test_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[test_vector]
y_test_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values[holdout_vector]
y_test_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values[holdout_vector]

In [ ]:
X_train.shape, X_test.shape, X_holdout.shape

((11197708, 41), (2798988, 41), (144925, 41))

In [ ]:
data_df[data_df['Id'].isin(train_id_df['id'])].to_csv('data_train.csv', index = False)
data_df[data_df['Id'].isin(test_id_df['id'])].to_csv('data_test.csv', index = False)
data_df[data_df['Id'].isin(holdout_id_df['id'])].to_csv('data_holdout.csv', index = False)

In [ ]:
print()
data_df = data_df[data_df[['W1', 'WX', 'W2', 'X1', 'X2', 'W12', 'Over', 'Under', 'H1', 'H2' ]].sum(axis = 1) > 0]
data_df = data_df[~(data_df['Active.1'] == 0)]

In [ ]:
len(data_df[data_df['Id'].isin(train_id_df['id'])]), \
len(data_df[data_df['Id'].isin(test_id_df['id'])]), \
len(data_df[data_df['Id'].isin(holdout_id_df['id'])])

(3375750, 849640, 124367)

In [ ]:
data_df[data_df['P1_transformed'].isna()]

In [ ]:
names = [
    'Id', 'StatTime', 'Minute', 'Active', 'Score1', 'Score2', 'A1', 'A2', 'DA1', 'DA2', 'Pos1', 'Pos2',
    'Off1', 'Off2', 'On1', 'On2', 'YC1', 'YC2', 'RC1', 'RC2', 'Sub1', 'Sub2', 'Pen1', 'Pen2',
    'Cor1', 'Cor2', 'Period', 'Comment', 'D', 'I', 'Active.1', 'Time', 'Minute.1', 'RawTime',
    'Score1.1', 'Score2.1', 'Period.1', 'W1', 'WX', 'W2', 'X1', 'X2', 'W12', 'TotalValue' ,'Over',
    'Under', 'Hand1Value', 'H1', 'H2' 
]


In [ ]:
data_train = data_train[data_train[['W1', 'WX', 'W2', 'X1', 'X2', 'W12', 'Over', 'Under', 'H1', 'H2' ]].sum(axis = 1) > 0]

In [ ]:
data_train[~(data_train['Active.1'] == 0)]

In [ ]:
data_train.to_csv('./data_train.csv.gz', index = False, compression = {'method':'gzip'})
data_test.to_csv('./data_test.csv.gz', index = False, compression = {'method':'gzip'})
data_holdout.to_csv('./data_holdout.csv.gz', index = False, compression = {'method':'gzip'})

In [ ]:
data_df = pd.read_csv(
    './data.csv', 
    sep = ';',
#    usecols = [col],
#    #parse_dates = ['StatTime'],
    header = None, 
    names = names, 
#    dtype = data_types_dict, 
#    nrows = 10_000
    skiprows = 10_000_000
    )

In [1]:
!wget -q -O ./data.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/dRWA_aIFU0AHGQ

In [6]:
!mkdir -p ./Result2

In [ ]:
!unrar e ./data.rar ./Result2/

In [10]:
!tail ./Result2/4054120.csv

4054120;28.06.2022 22:37:00;81;1;1;0;70;72;64;46;51;49;4;2;6;1;2;2;0;0;4;4;0;0;5;5;2;;28.06.2022 22:36:58;28.06.2022 20:59:57;13899844;1;28.06.2022 22:36:31;81;80:00;1;0;2;;;1.132;6.6;34;;5.55;1.097;1.5;2.26;1.71;-1;1.63;2.45
4054120;28.06.2022 22:38:00;82;1;1;0;70;72;66;46;51;49;4;2;6;1;2;2;0;0;4;4;0;0;5;5;2;;28.06.2022 22:37:59;28.06.2022 20:59:57;13899844;1;28.06.2022 22:37:39;82;81:15;1;0;2;;;1.124;6.8;36;;5.75;1.091;1.5;2.375;1.65;-1;1.63;2.45
4054120;28.06.2022 22:39:00;83;1;1;0;70;72;66;47;51;49;4;2;6;1;2;2;0;0;4;4;0;0;5;6;2;;28.06.2022 22:38:46;28.06.2022 20:59:57;13899844;1;28.06.2022 22:38:41;83;82:15;1;0;2;;;1.115;7.1;38;;6;1.084;1.5;2.51;1.59;-1;1.63;2.45
4054120;28.06.2022 22:40:00;84;1;1;0;71;72;67;49;51;49;4;2;6;1;2;2;0;0;4;4;0;0;5;6;2;;28.06.2022 22:39:47;28.06.2022 20:59:57;13899844;1;28.06.2022 22:39:47;84;83:15;1;0;2;;;1.106;7.4;41;;6.3;1.077;1.5;2.68;1.53;-1;1.64;2.43
4054120;28.06.2022 22:41:00;85;1;1;0;72;73;68;49;51;49;4;2;6;1;2;2;0;0;5;4;0;0;5;6;2;;28.06.2022 22

#### Predefine data

In [11]:
import datetime
data_types_dict = {
    'Id':np.int32, 
    'StatTime':str, 
    'Minute':np.int8, 
    'Active': np.int8, 'Score1':np.int8, 'Score2':np.int8,
    'A1':np.int16, 'A2':np.int16, 'DA1':np.int16, 'DA2':np.int16, 'Pos1':np.float32, 'Pos2':np.float32,
    'Off1':np.int8, 'Off2':np.int8, 'On1':np.int8, 'On2':np.int8, 'YC1':np.int8, 'YC2':np.int8,
    'RC1':np.int8, 'RC2':np.int8, 'Sub1':np.int8, 'Sub2':np.int8, 'Pen1':np.int8, 'Pen2':np.int8,
    'Cor1':np.int8, 'Cor2':np.int8, 'Period':np.int8, 
    'D':np.datetime64,
    'I':np.int32, 'Active.1':np.int8, 
    'Time':np.datetime64, 
    'Minute.1':np.int8, 
    'RawTime':np.datetime64, 
    'Score1.1':np.int8, 'Score2.1':np.int8, 'Period.1':np.int8, 
    'W1':np.float16, 'WX':np.float16, 'W2':np.float16, 'X1':np.float16, 'X2':np.float16, 'W12':np.float16, 'TotalValue':np.float16, 
    'Over':np.float16, 'Under':np.float16, 'Hand1Value':np.float16, 'H1':np.float16, 'H2':np.float16 
}

In [12]:
k_cols = [
    'W1', 'WX', 'W2', 'X1', 'X2', 'W12', 'TotalValue' ,'Over',
    'Under', 'Hand1Value', 'H1', 'H2' 
]

In [13]:
drop_cols = [
    #'StatTime', 
    'Comment', 'D', 'I', 'Time', 'Minute.1', 'RawTime',
    'Score1.1', 'Score2.1', 'Period.1', 'Period', 'Periods', 'Serve'
]

In [26]:
from glob import glob
from tqdm import tqdm
import functools

In [19]:
def define_files_list(data_start:str, folder:str, date_end = '') -> list:
    '''
    data_start, data_end
    folder: ./folder/
    '''
    if date_end == '':
        return sorted(
            [file 
            for file in glob(folder + '*') 
            if int(file.split('/')[-1].split('.csv')[0]) >= int(date_start)]
            )
    else:
        return sorted(
            [file 
            for file in glob(folder + '*') 
            if int(file.split('/')[-1].split('.csv')[0]) >= int(date_start) & \
                int(file.split('/')[-1].split('.csv')[0]) <= int(date_end)]
            )

#### _

In [14]:
info_cols = ['Id', 'Result1', 'Result2']
prem_cols = ['Id', 'P1', 'PX', 'P2']
res_cols = [cols for cols in data_types_dict if cols not in drop_cols]

info_folder = './Index/'
prem_folder = './Prem/'
data_folder = './Result2/'

In [ ]:
pd.read_csv('./Result2/4054120.csv', sep = ';')

In [23]:
### собрать датасет с даты:
year_start = '2022'
month_start = '01'
day_start = '01'
### до даты
year_end = ''
month_end = ''
day_end = ''
date_start = year_start + month_start + day_start
date_end = year_end + month_end + day_end

In [32]:
data_df = pd.concat(map(functools.partial(
    pd.read_csv, 
    sep = ';',
    parse_dates = ['StatTime'],
    usecols = res_cols
                                    ), glob('/content/Result2/*.csv')), ignore_index=True)

In [34]:
data_df.head(100)

,Id,StatTime,Minute,Active,Score1,Score2,A1,A2,DA1,DA2,Pos1,Pos2,Off1,Off2,On1,On2,YC1,YC2,RC1,RC2,Sub1,Sub2,Pen1,Pen2,Cor1,Cor2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2
0,5490795,2022-08-20 09:37:00,1,1,0,0,0,1,0,0,NaN,NaN,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5490795,2022-08-20 09:38:00,2,1,0,0,2,3,1,1,44.0,56.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5490795,2022-08-20 09:39:00,2,1,0,0,3,4,1,2,41.0,59.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5490795,2022-08-20 09:40:00,4,1,0,0,6,6,3,3,47.0,53.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5490795,2022-08-20 09:41:00,5,1,0,0,8,7,4,4,51.0,49.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5490795,2022-08-20 09:42:00,5,1,0,0,8,7,4,5,52.0,48.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5490795,2022-08-20 09:43:00,7,1,0,0,10,9,4,6,48.0,52.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5490795,2022-08-20 09:44:00,8,1,0,0,11,12,4,10,42.0,58.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5490795,2022-08-20 09:45:00,9,1,0,0,13,13,6,11,43.0,57.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5490795,2022-08-20 09:46:00,10,1,0,0,14,15,6,12,43.0,57.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
